# BA3H Reconstruct a String from its k-mer Composition

In [84]:
from collections import defaultdict
import random
import copy

def explored(graphDict):
    totalValues = sum([len(graphDict[k]) for k in graphDict])
    if totalValues == 0:
        return True
    return False

def deBrujin(patterns):
    nodes = defaultdict(list)
    for i in patterns:
        nodes[i[:-1]].append(i[1:])
    return(nodes)
        
def eulerCycle(graphDict):
    startNode = random.choice(list(graphDict))
    currNode = startNode
    graphCopy = copy.deepcopy(graphDict)
    visited = []
    while len(graphCopy[currNode]) > 0:
            visited.append(currNode)
            nextNode = random.choice(graphCopy[currNode])
            graphCopy[currNode].remove(nextNode)
            currNode = nextNode
            if nextNode == startNode:
                break
    while not explored(graphCopy):
        #reorder visited
        newStartNode = [k for k in visited if len(graphCopy[k]) > 0]
        idx = visited.index(newStartNode[0])
        visited = visited[idx:] + visited[:idx]
        currNode = newStartNode[0]
        while len(graphCopy[currNode]) > 0:
            visited.append(currNode)
            nextNode = random.choice(graphCopy[currNode])
            graphCopy[currNode].remove(nextNode)
            currNode = nextNode
            if nextNode == startNode:
                break
    visited.append(currNode)
    return(visited)

def inDegree(k, graphDict):
    out = 0
    for i in graphDict.keys():
        if i == k:
            continue
        else:
            if k in graphDict[i] : out += 1
    return(out)

def addEdge(graphDict):
    #add edge to convert a semi-balanced graph into a balaced graph
    allKeys = graphDict.keys()
    allValues = []
    for v in graphDict.values():
        allValues.extend(v)
    outNode = [k for k in graphDict.keys() if len(graphDict[k]) < inDegree(k, graphDict)]
    outNode.extend([k for k in allValues if k not in allKeys])
    inNode = [k for k in graphDict.keys() if len(graphDict[k]) > inDegree(k, graphDict)]
    return inNode,outNode
    
def eulerPath(graphDict):
    # @param graphDict: A dict of lists, keys are source nodes and values are target nodes
    # built upon eulerCycle
    inNode, outNode = addEdge(graphDict)
    graphDict[outNode[0]].extend(inNode)
    cycle = eulerCycle(graphDict)
    #remove the last node which was added to form a eulerCycle
    #if the last node is the inNode, remove first node (in this case, first node would be inNode as well)
    #so remove first node to avoid out of index error in idx
    if cycle[0] == inNode[0]:
        cycle = cycle[1:]
    else:
        cycle = cycle[:-1]
    idx = [x for x in range(len(cycle) - 1) if cycle[x] == outNode[0] and cycle[x + 1] == inNode[0]]
    cycle = cycle[idx[0] + 1:] + cycle[:idx[0] + 1]
    genome = cycle[0] + ''.join([cycle[i][-1] for i in range(1, len(cycle), 1)])
    return(genome)


In [93]:
def BA3H(filename):
    with open(filename) as f:
        k = int(f.readline().rstrip())
        pattern = []
        for line in f:
            pattern.append(line.rstrip())
        graphDict = deBrujin(pattern)
        genome = eulerPath(graphDict)
        return(genome)

## Test

In [86]:
test = ['CTTA', 'ACCA', 'TACC', 'GGCT', 'GCTT', 'TTAC']
testGraph = deBrujin(test)
eulerPath(testGraph)

'GGCTTACCA'

In [94]:
BA3H('BA3H-test.txt')

'GGCTTACCA'

In [95]:
BA3H('BA3H-test2.txt')

'TGCCCCTTTGATCGCGGTTCTCGAATCCATGTAAATACAAAGATCTTATGTCCGCCGCGTATAGCGGTCGTAAAAATCTACGAGTTTCGATAACTCCAGGATCAATGCGGAACTATGCCCTTATAATAAGGCCACAATTAGTGCGCGTATTAGTGCGATTCCCATTTGCTCCTTTTCTCAACGACCAACGTAGGCGGGGGATGAGTATGCACACGCCCACCCGCTACACTCGACCCTCTCGGCTCTTTTTGTACCGGGGGCCTATATCTCCTGCACCGCCACCATCGCGTTCTCTCTTATTTTGCTATTATTATTCTTTCCAGAACATATGACATATCAGTGCAAGCTGAATCGCGAAGCGGCACTTAATACGATTTCTTGCGATGTGTCTTCTCGCGGCAATTGCTAGTGCCTGGTAAGTCACCGTGATCGTGTCTATG'

## Quiz

In [96]:
BA3H('rosalind_ba3h.txt')

'CCTAAGACTTCTAGCCATAGTGTCGGTATTGGGGTAGACTGATTTATTAGGCCCTTGGATAACGATTACCCGGGCGACAGATGTAGGATGAGGATATTGTGGTTGCGAGACGGAGGGTATGGGTCAACTTCCAAGCATTTGCTCCAAGCGCTGCTCGAGCCCTGATTGCACTAGCTATTGATACATATACCGGTGACATACAGCGGGTCGAGCTTTGTGACACACGTCTTTATACGCTCGGGTATCAATATTCCGGCATGGGAACCCTGACGGGTTTTGAAACCACGTTGTTGCCATTAATTGTGTATGTGTCGGACCTGTAAGTCCCTCAGTGCTTGCGTGCCATGGCAGTGCGCATCGTACGTATAAGCCGGCTTCCGATTCAGCGTCCTGCACAGTTCCAATGATTACGGGTCTATAGGACCTTCCATTTCGCGTGGTCGGAAGGGGGGACTAACTGTGTAGGTGGTGGGGGAAGTGGCACAACCAAATCTCACCAACTACCCGCTTCGAAAGCCGCTATTAAACGTGCGAACCTTCGTCACCTAACTATTCCTATTTAGTGCACATTCAGGCTTACTAAGAGAGGCAGCGAAACGTCTTCCATACTCCTACCAGGAGTCCTTATAAATTTCGCGGACTTGACACCTGTTCAATTGTCCGTCCCTCCCTCTGAATCTCCCCGGATGAGTAACTCGCACGGTGTTACCGCCCTTTACTTAGTTGCCCAATTGCACAGTGGGCGATAATTCGCGGGTTGTGATCTTTGGGCAAGTAGAATCGTCAGAGCCCGGGGAGTTCCGTACCTGTTTAAACCCTGGGTGGACCACATTCTCTCAAACCATGGCTAGATACCCCATTGGCACGCGGGTTATAACGGTCCTACTACCTCGTATTCCAGCAGACTGGTTTCCTCAATCACTTTTCGCCTATCTAGCGGTACTGCGCGGTAACGTTGAGGGCGTTTTAATTAATTCCCCGTACGGGTTGTTCCAATTA